In [102]:
import cv2
import numpy as np
import os

Project demonstrating the pipeline for camera calibration using the chessboard method. We were allowed to use any function to do this, so I'm not too interested with it right now. Maybe later I will solve it by hand using Zhangs method to decompose H into the K, R matrices. I would need to learn how to handle distortion however.

The opencv docs are here 
https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

In [103]:
def gen_world_pts():
 
    world_pts = []

    # converting mm to m and generating world points
    for x in range(6):
        for y in range(9):
            xw = 0.0215*x
            yw = 0.0215*y
            
            world_pts.append([xw, yw])

    world_pts = np.array(world_pts)
    return world_pts

In [111]:
image_pts = np.array([])
world_pts = np.array([])
num_images = 0

for fname in os.listdir('./Calibration_Imgs/')[:1]:
        
    image = cv2.imread(f"./Calibration_Imgs/{fname}")

    ret, image_corners = cv2.findChessboardCorners(image, (9,6))
    
    if ret == True:
        num_images += 1
        
        image_corners = image_corners.reshape(-1, 2)
        
        if len(image_pts) != 0:
            image_pts = np.concatenate((image_pts, image_corners))
            world_pts = np.concatenate((world_pts, gen_world_pts()))
        else:
            image_pts = image_corners
            world_pts = gen_world_pts()


In [113]:
for i in image_pts:
    print(i)

[ 362.04318 1477.383  ]
[ 374.50928 1353.3971 ]
[ 387.00433 1234.4147 ]
[ 398.75333 1121.3866 ]
[ 409.5982 1012.0066]
[420.47037 906.1987 ]
[430.96402 805.1585 ]
[441.65222 708.3719 ]
[451.51297 614.435  ]
[ 494.35486 1482.1812 ]
[ 504.624  1358.8696]
[ 514.13806 1240.5941 ]
[ 522.98083 1126.5211 ]
[ 531.5785 1017.6117]
[539.6976  911.65686]
[548.2362 809.6145]
[556.26776 712.3994 ]
[564.42084 618.7465 ]
[ 625.62115 1486.3792 ]
[ 632.9711 1363.3252]
[ 639.91113 1245.0726 ]
[ 646.4683 1130.983 ]
[ 652.59296 1021.5931 ]
[658.9867 916.1094]
[664.6223 813.7479]
[670.753  716.3617]
[676.3825 622.7387]
[ 756.5799 1489.7246]
[ 761.0999 1367.0531]
[ 765.3668 1249.4617]
[ 769.7625 1135.3274]
[ 773.9579 1025.5602]
[777.65454 919.545  ]
[781.84705 817.5821 ]
[785.2827 720.5441]
[789.234  627.4801]
[ 889.038  1494.3754]
[ 890.24023 1370.2596 ]
[ 891.77905 1252.8568 ]
[ 893.74396 1139.7369 ]
[ 895.54285 1030.5911 ]
[897.55316 924.5502 ]
[899.01776 822.1933 ]
[900.7073 724.6734]
[901.6839  631.68176

In [114]:
def find_h(image_points, world_points):
    
    a_mat = np.array([])
    rows = []
    
    for i in range(len(image_points)):
        print(i)
        xc = image_points[i][0]
        yc = image_points[i][1]
        
        Xw = world_points[i][0]
        Yw = world_points[i][1]

        rows.append([Xw, Yw, 1, 0,  0,  0, -xc*Xw, -xc*Yw, -xc])
        rows.append([0,  0,  0, Xw, Yw, 1, -yc*Xw, -yc*Yw, -yc])
        
    
    a_mat = np.array(rows)
    aTa = a_mat.T@a_mat
    eigvals, eigvecs = np.linalg.eig(aTa)
    
    p = eigvecs[:, np.argmin(eigvals)]
    p = p.reshape(3, 3)
    return p

In [115]:
image_pts

array([[ 362.04318, 1477.383  ],
       [ 374.50928, 1353.3971 ],
       [ 387.00433, 1234.4147 ],
       [ 398.75333, 1121.3866 ],
       [ 409.5982 , 1012.0066 ],
       [ 420.47037,  906.1987 ],
       [ 430.96402,  805.1585 ],
       [ 441.65222,  708.3719 ],
       [ 451.51297,  614.435  ],
       [ 494.35486, 1482.1812 ],
       [ 504.624  , 1358.8696 ],
       [ 514.13806, 1240.5941 ],
       [ 522.98083, 1126.5211 ],
       [ 531.5785 , 1017.6117 ],
       [ 539.6976 ,  911.65686],
       [ 548.2362 ,  809.6145 ],
       [ 556.26776,  712.3994 ],
       [ 564.42084,  618.7465 ],
       [ 625.62115, 1486.3792 ],
       [ 632.9711 , 1363.3252 ],
       [ 639.91113, 1245.0726 ],
       [ 646.4683 , 1130.983  ],
       [ 652.59296, 1021.5931 ],
       [ 658.9867 ,  916.1094 ],
       [ 664.6223 ,  813.7479 ],
       [ 670.753  ,  716.3617 ],
       [ 676.3825 ,  622.7387 ],
       [ 756.5799 , 1489.7246 ],
       [ 761.0999 , 1367.0531 ],
       [ 765.3668 , 1249.4617 ],
       [ 7

In [116]:
world_pts

array([[0.    , 0.    ],
       [0.    , 0.0215],
       [0.    , 0.043 ],
       [0.    , 0.0645],
       [0.    , 0.086 ],
       [0.    , 0.1075],
       [0.    , 0.129 ],
       [0.    , 0.1505],
       [0.    , 0.172 ],
       [0.0215, 0.    ],
       [0.0215, 0.0215],
       [0.0215, 0.043 ],
       [0.0215, 0.0645],
       [0.0215, 0.086 ],
       [0.0215, 0.1075],
       [0.0215, 0.129 ],
       [0.0215, 0.1505],
       [0.0215, 0.172 ],
       [0.043 , 0.    ],
       [0.043 , 0.0215],
       [0.043 , 0.043 ],
       [0.043 , 0.0645],
       [0.043 , 0.086 ],
       [0.043 , 0.1075],
       [0.043 , 0.129 ],
       [0.043 , 0.1505],
       [0.043 , 0.172 ],
       [0.0645, 0.    ],
       [0.0645, 0.0215],
       [0.0645, 0.043 ],
       [0.0645, 0.0645],
       [0.0645, 0.086 ],
       [0.0645, 0.1075],
       [0.0645, 0.129 ],
       [0.0645, 0.1505],
       [0.0645, 0.172 ],
       [0.086 , 0.    ],
       [0.086 , 0.0215],
       [0.086 , 0.043 ],
       [0.086 , 0.0645],


In [117]:
h = find_h(image_pts, world_pts)
h

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


array([[-7.89293254e-01, -1.23415797e-01, -4.65821023e-02],
       [-2.83702950e-02,  5.68036233e-01, -1.90117700e-01],
       [-7.30363581e-07, -1.25946057e-04, -1.28706277e-04]])

In [118]:
def get_reproj_err(image_points, world_pts, p):
    
    ones = np.ones(shape=(len(world_pts), 1))
    homogeneous_world_pts = np.concatenate((world_pts, ones), axis=1)
    reproj_pts = np.zeros_like(image_points).astype(np.float32)
        
    for i in range(len(homogeneous_world_pts)):
        w_pt = homogeneous_world_pts[i]
    
        r_pt = p@w_pt
        r_pt = r_pt/r_pt[-1]
        reproj_pts[i] = r_pt[:-1]

    reproj_err = np.sqrt(np.sum((reproj_pts-image_pts)**2, axis = 1))

    return reproj_err

In [119]:
reproj_err = get_reproj_err(image_pts, world_pts, h)
np.average(reproj_err)

0.74556816